In [32]:
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, SignalStrategy, TrailingStrategy
# from backtesting.test import SMA
from ta import momentum
import pandas as pd
import datetime as dt


def SMA(arr, n):
    return pd.Series(arr).rolling(n).mean()


def EMA(arr, n):
    return pd.Series(arr).ewm(span=n, adjust=False).mean()


def RSI_Indicator(arr, n):
    arr_series = pd.Series(arr)
    rsi_indicator = momentum.RSIIndicator(arr_series, window=n)
    return rsi_indicator.rsi()


class RsiStrategy(Strategy):
    def init(self):
        # Load the price and RSI
        price = self.data.Close
        self.rsi = self.I(RSI_Indicator, price, 14)

    def next(self):
        # Buy when RSI crosses above 30
        if crossover(self.rsi, 30):
            self.buy()
        # Sell when RSI crosses below 70
        elif crossover(70, self.rsi):
            self.sell()


class SmaCross(Strategy):
    n1 = 10
    n2 = 20
    
    def init(self):
        price = self.data.Close
        # SMA = pd.Series(arr).rolling(n).mean()
        self.ma1 = self.I(SMA, price, self.n1)
        self.ma2 = self.I(SMA, price, self.n2)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


# symbol = 'AAPL'
symbol = 'GOOG'

# Define the start and end dates for the data
# start_date = '2022-01-11'
# end_date = '2023-01-01'

# set current date as end date
end_date = dt.datetime.now().strftime('%Y-%m-%d')

# Download the data using yfinance
data = yf.download(symbol, start="2021-01-01", end=end_date, interval='1d')


bt = Backtest(data, SmaCross, commission=.002,
              cash=2_000,
              exclusive_orders=True)
bt.run()

stats = bt.optimize(n1=range(5, 50, 1),
                    n2=range(10, 100, 1),
                    maximize='Win Rate [%]',
                    constraint=lambda param: param.n1 < param.n2)

bt.plot()
stats


[*********************100%***********************]  1 of 1 completed


/home/da2343/.conda/envs/cs685/lib/python3.9/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 3230 configurations.
  output = _optimize_grid()
/home/da2343/.conda/envs/cs685/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/da2343/.conda/envs/cs685/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-01-04 00:00:00
End                       2023-03-24 00:00:00
Duration                    809 days 00:00:00
Exposure Time [%]                        52.5
Equity Final [$]                  2618.108367
Equity Peak [$]                    2630.58847
Return [%]                          30.905418
Buy & Hold Return [%]               22.737576
Return (Ann.) [%]                   12.883621
Volatility (Ann.) [%]               31.176213
Sharpe Ratio                         0.413252
Sortino Ratio                        0.697186
Calmar Ratio                         0.578985
Max. Drawdown [%]                  -22.252079
Avg. Drawdown [%]                   -7.872618
Max. Drawdown Duration      132 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                    6
Win Rate [%]                        83.333333
Best Trade [%]                      17.046291
Worst Trade [%]                    -10.574129
Avg. Trade [%]                    

In [34]:
stats._strategy
bt.plot(plot_volume=False, plot_pl=False)

/home/da2343/.conda/envs/cs685/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/home/da2343/.conda/envs/cs685/lib/python3.9/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p26230', ...)

In [35]:
stats.tail()

Expectancy [%]                                             5.125561
SQN                                                        1.200019
_strategy                                     SmaCross(n1=40,n2=98)
_equity_curve                      Equity  DrawdownPct DrawdownD...
_trades              Size  EntryBar  ExitBar  EntryPrice   ExitP...
dtype: object

In [36]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-15,266,419,125.775441,109.199997,248.631666,0.131786,2022-01-24,2022-09-01,220 days
1,20,419,420,109.418397,111.339996,38.431988,0.017562,2022-09-01,2022-09-02,1 days
2,-20,420,424,111.117316,110.050003,21.346266,0.009605,2022-09-02,2022-09-09,7 days
3,20,424,435,110.270103,98.610001,-233.202049,-0.105741,2022-09-09,2022-09-26,17 days
4,-21,435,542,98.412781,90.160004,173.308316,0.083859,2022-09-26,2023-03-01,156 days
5,24,542,559,90.340324,105.739998,369.592181,0.170463,2023-03-01,2023-03-24,23 days
